In [1]:
import transformers
from datasets import concatenate_datasets
from datasets import load_from_disk, load_metric
from transformers import AutoTokenizer
from utils_qa import postprocess_qa_predictions
from trainer_qa import QuestionAnsweringTrainer
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer,EvalPrediction
from transformers import default_data_collator
import os
max_length = 512
doc_stride = 128
batch_size = 4

model_pretrain = "bert-large-cased-whole-word-masking-finetuned-squad" #'roberta-large' 

dir_name = 'BERT_SQUAD_TOK_L_REL'
'''
cp = 'checkpoint-1000'
save_dir = os.path.join("/storage/model", dir_name)
checkpoint = os.path.join(save_dir,cp)

if 'REL' in dir_name:
    model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)#, position_embedding_type="relative_key_query"
else:
    model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)

#model = AutoModelForQuestionAnswering.from_pretrained(model_pretrain)
'''
tokenizer = AutoTokenizer.from_pretrained(
    model_pretrain,
    use_fast=True,
)
if 'TOK'in dir_name:
    tokenizer.add_tokens(['<P>','</P>', '<Table>','</Table>','<Li>','</Li>','<Th>','</Th>','<Td>','</Td>','Ul','/Ul'],special_tokens=True)
pad_on_right = tokenizer.padding_side == "right"

'''
from datasets import load_dataset
dataset_name = 'squad'
datasets = load_dataset(dataset_name)
train_examples = datasets['train']
train_dataset = train_examples.map(prepare_validation_features, batched=True, remove_columns=train_examples.column_names)
eval_examples = datasets['validation'].select(range(max_val_samples))
eval_dataset = eval_examples.map(prepare_validation_features, batched=True, remove_columns=eval_examples.column_names)

#train_dataset=load_from_disk("/storage/{}/train_{}_{}".format(dir_name, 20000,15950)).shuffle()

eval_examples = load_from_disk("/storage/{}/val_example".format(dir_name))
eval_examples = eval_examples.select(range(max_val_samples))
eval_dataset = eval_examples.map(prepare_validation_features, batched=True, remove_columns=eval_examples.column_names)
eval_dataset = eval_dataset.select(range(max_val_samples))
#eval_dataset = load_from_disk("/storage/{}/val".format(dir_name))
'''

'\nfrom datasets import load_dataset\ndataset_name = \'squad\'\ndatasets = load_dataset(dataset_name)\ntrain_examples = datasets[\'train\']\ntrain_dataset = train_examples.map(prepare_validation_features, batched=True, remove_columns=train_examples.column_names)\neval_examples = datasets[\'validation\'].select(range(max_val_samples))\neval_dataset = eval_examples.map(prepare_validation_features, batched=True, remove_columns=eval_examples.column_names)\n\n#train_dataset=load_from_disk("/storage/{}/train_{}_{}".format(dir_name, 20000,15950)).shuffle()\n\neval_examples = load_from_disk("/storage/{}/val_example".format(dir_name))\neval_examples = eval_examples.select(range(max_val_samples))\neval_dataset = eval_examples.map(prepare_validation_features, batched=True, remove_columns=eval_examples.column_names)\neval_dataset = eval_dataset.select(range(max_val_samples))\n#eval_dataset = load_from_disk("/storage/{}/val".format(dir_name))\n'

In [2]:
from utils_data import read_annotation_gzip
from datasets import Dataset
max_val_samples= 20
path = '/storage/datset/v1.0_sample_nq-dev-sample.jsonl.gz'
dic = read_annotation_gzip(path)
eval_examples = Dataset.from_dict(dic).select(range(max_val_samples))

In [4]:
eval_dataset = eval_examples.map(prepare_validation_features, batched=True, remove_columns=eval_examples.column_names)
for key in eval_dataset[0]:
    print(key)


['5225754983651766092', '6986236841860957647', '-3290814144789249484', '5745452844331879752', '8851020722386421469', '-909916459264223411', '7524133268429333037', '-7660771254611710392', '3223058827565794274', '8467542931261548456', '-8817510910624892182', '8081436745274892553', '955374967862684316', '-4340755100872459608', '-4752044886865067782', '3368255430684131446', '-360833956386934695', '3221262508309669486', '-3632974700795137148', '6117400998512150504']
attention_mask
end_positions
example_id
input_ids
offset_mapping
start_positions
token_type_ids


In [5]:
metric = load_metric("squad_v2")
def compute_metrics(p: EvalPrediction):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

data_collator = default_data_collator

for epoch in range(6200,9800,200):
    cp = 'checkpoint-{}'.format(epoch)
    save_dir = os.path.join("/storage/model", dir_name)
    checkpoint = os.path.join(save_dir,cp)

    model = AutoModelForQuestionAnswering.from_pretrained(checkpoint)
    # Initialize our Trainer
    print(epoch)
    
    args = TrainingArguments(
        save_dir,
        learning_rate=3e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=64,
        num_train_epochs=3,
        save_steps = 600,
        eval_steps = 600,
        evaluation_strategy ='steps',
        gradient_accumulation_steps=8,
    )
    trainer = QuestionAnsweringTrainer(
        model=model,
        args=args,
        train_dataset=None,
        eval_dataset=eval_dataset,
        eval_examples=eval_examples,
        tokenizer=tokenizer,
        data_collator=data_collator,
        post_process_function=post_processing_function,
    )

    metrics = trainer.evaluate()
#metrics["eval_samples"] = min(max_val_samples, len(eval_dataset))
#trainer.log_metrics("eval", metrics)
#trainer.save_metrics("eval", metrics)

6200



Saving predictions to /storage/model/BERT_SQUAD_TOK_L_REL/eval_predictions.json.
Saving nbest_preds to /storage/model/BERT_SQUAD_TOK_L_REL/eval_nbest_predictions.json.
{'eval_loss': 0.5476325750350952}
6400



Saving predictions to /storage/model/BERT_SQUAD_TOK_L_REL/eval_predictions.json.
Saving nbest_preds to /storage/model/BERT_SQUAD_TOK_L_REL/eval_nbest_predictions.json.
{'eval_loss': 0.6491122245788574}
6600



Saving predictions to /storage/model/BERT_SQUAD_TOK_L_REL/eval_predictions.json.
Saving nbest_preds to /storage/model/BERT_SQUAD_TOK_L_REL/eval_nbest_predictions.json.
{'eval_loss': 0.3207642436027527}
6800



Saving predictions to /storage/model/BERT_SQUAD_TOK_L_REL/eval_predictions.json.
Saving nbest_preds to /storage/model/BERT_SQUAD_TOK_L_REL/eval_nbest_predictions.json.
{'eval_loss': 0.5254642963409424}
7000


...


Saving predictions to /storage/model/BERT_SQUAD_TOK_L_REL/eval_predictions.json.
Saving nbest_preds to /storage/model/BERT_SQUAD_TOK_L_REL/eval_nbest_predictions.json.
{'eval_loss': 0.4368992745876312}
9000



Saving predictions to /storage/model/BERT_SQUAD_TOK_L_REL/eval_predictions.json.
Saving nbest_preds to /storage/model/BERT_SQUAD_TOK_L_REL/eval_nbest_predictions.json.
{'eval_loss': 0.701496958732605}
9200



Saving predictions to /storage/model/BERT_SQUAD_TOK_L_REL/eval_predictions.json.
Saving nbest_preds to /storage/model/BERT_SQUAD_TOK_L_REL/eval_nbest_predictions.json.
{'eval_loss': 0.4135887920856476}
9400



Saving predictions to /storage/model/BERT_SQUAD_TOK_L_REL/eval_predictions.json.
Saving nbest_preds to /storage/model/BERT_SQUAD_TOK_L_REL/eval_nbest_predictions.json.
{'eval_loss': 0.47856900095939636}
9600



Saving predictions to /storage/model/BERT_SQUAD_TOK_L_REL/eval_predictions.json.
Saving nbest_preds to /storage/model/BERT_SQUAD_TOK_L_REL/eval_nbest_predictions.json.
{'eval_loss': 0.4144849181175232}


In [9]:
print(metrics)

{'eval_loss': 0.38271820545196533}


In [3]:
# Validation preprocessing
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    #examples = simplify_nq_example(examples)
    
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    offset_mapping = tokenized_examples["offset_mapping"]

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)
        
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)           
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

        ## TODO for gz file
        start_char, end_char = -1, -1
        # Start/end character index of the answer in the text.
        for candidate in examples["annotations"][sample_index]:
            cur = candidate["long_answer"]["start_char"]
            if cur != -1:
                start_char = cur
                end_char = candidate["long_answer"]["end_char"]
        # If no answers are given, set the cls_index as answer.
        if start_char == -1:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)
                
    return tokenized_examples

# Training preprocessing
def prepare_train_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.

    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        ## TODO for gz file
        start_char, end_char = -1, -1
        # Start/end character index of the answer in the text.
        for candidate in examples["annotations"][sample_index]:
            cur = candidate["long_answer"]["start_char"]
            if cur != -1:
                start_char = cur
                end_char = candidate["long_answer"]["end_char"]
        # If no answers are given, set the cls_index as answer.
        if start_char == -1:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples


from utils_qa import postprocess_qa_predictions
# Post-processing:
def post_processing_function(examples, features, predictions, stage="eval"):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        n_best_size=20,
        max_answer_length=500,
        output_dir= args.output_dir,
        is_world_process_zero=trainer.is_world_process_zero(),
        prefix=stage,
        version_2_with_negative=True,
    )
    # Format the result to the format the metric expects.
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]
    #references = [{"id": ex["id"], "answers": ex[answer_column_name]} for ex in examples]

    references = [{"id": ex['id'], 'start_token': ex['annotations'][0]['long_answer']['start_token'], 
                   'end_token': ex['annotations'][0]['long_answer']['end_token']
      } for ex in examples ]

    return EvalPrediction(predictions=formatted_predictions, label_ids=references)



In [2]:
import random
random.randomInt()

AttributeError: module 'random' has no attribute 'randomInt'